<h1 dir=rtl>eothello</h1>
<div dir=rtl>
بازی اوتلو قرار است در یک صفحه 8*8 بین انسان و هوش مصنوعی پیاده سازی شود. 
برای نمایش موقعیت هر خانه از صفحه قرار داد زیر را تعریف میکنیم:

BLACK = -1

WHITE = 1

EMPTY = 0

یعنی اگر یک باشد در خانه مهره سفید است اگر صفر باشد خانه خالی است و اگر منفی یک باشد در خانه مهره سیاه قرار دارد.این قرارداد علامتی ساده باعث می‌شود عملیات های مورد نیاز ما راحت شوند (مثلاً board[x][y] * player > 0 یعنی مهره متعلق به همان بازیکن است).

صفحه معمولاً به صورت آرایهٔ ۲بعدی تعریف می‌شود و موقعیت اولیهٔ اوتلو این‌طور است:(ایندکس ها در صفحه از 0 تا 7 شماره گذاری میشوند)

 (3,3)=WHITE, (3,4)=BLACK, (4,3)=BLACK, (4,4)=WHITE

 تابع initial_board صفحه اولیه را برای ما چاپ میکند.

In [ ]:
EMPTY, BLACK, WHITE = 0, -1, 1

def initial_board():
    board = [[EMPTY for _ in range(8)] for _ in range(8)]
    board[3][3], board[4][4] = WHITE, WHITE
    board[3][4], board[4][3] = BLACK, BLACK
    return board
board=initial_board()

<div dir=rtl>
تابع print_board .صفحه بازی را چاپ میکند 

عملکرد این تابع به این صورت است که روی آرایه برد(که یک ارایه  دو بعدی است) حرکت میکند و هر جا که مقدار خانه سیار بود داره مشکی، سفید بود دایره ی تو خالی و اگر خانه خالی بود نقطه را چاپ میکند.

In [4]:
def print_board(board):
    print("  " + " ".join(str(i) for i in range(8)))
    for i, row in enumerate(board):
        print(i, end=" ")
        for cell in row:
            if cell == BLACK:
                print("●", end=" ")
            elif cell == WHITE:
                print("○", end=" ")
            else:
                print(".", end=" ")
        print()

print_board(board)

  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . ○ ● . . . 
4 . . . ● ○ . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 


<div dir=rtl>
حال نوبت به تعریف حرکات معتبر در بازی میرسد که در تابع زیر تعریف و بررسی میشوند.

در بازی اوتلو، یک حرکت معتبر زمانی است که یک بازیکن مهره خود را در خانه‌ای قرار می‌دهد که یک یا چند مهره حریف را در یک یا چند راستا (افقی، عمودی یا مورب) محاصره کند. مهره‌های محاصره شده سپس به رنگ مهره بازیکن در می‌آیند

 لیستی از جهت های هشت گانه که مهره میتواند در آنها قرار بگیرد را در آرایه دایرکشن ذخیره میکنیم. 
 الگوریتم به این صورت است که روی برد حرکت میکنیم اگر خانه خالی بود به ازای هر جهت در دایرکشنز می آییم و متغیر های nx , ny را میسازیم که تاپل حاصل از این دو متغیر خانه همسایه را مشخص میکند.

 found_opponent این متغیر به این معنی است که ایا مهره حریف را دیدیم یا نه.

حال ابتدا در حلقه وایل بررسی میکنیم که خانه همسایه ای که بدست آوردیم   آیا در صفحه قرار دارد یا نه. سپس اگر خانه همسایه متعلق به رقیق بود متغیر فاند رو مقدارش را برابر صحیح قرار میدهیم.  و  سپس به مقدای ان ایکس و ان ایگرگ به اندازه جهتی که داریم درش جحکت میکنیم پیش میریم. انقدر این کار را ادامه می دهیم تا در نهایت یا همسایه خانه خالی شود یا به یک خانه متعلق به خود بازیکن برسیم. در این صورت اگر تا به حالت مهره ای از رقیب را دیده بودیم  حرکت را به لیست حرکت ها اضافه میکنیم.

و درنهایت چون ممکن است چند حرکت با چند جهت مختلف بدست بیایند در آرایه موو از ست استفاده میکنیم

مثال:

ردیف 3:  . W W W B . . .

حالا بررسی حرکت در (3,0)

جهت راست (dx=0, dy=1):

همسایه اول (3,1) = W -> دیدیم مهرهٔ حریف → found_opponent = True

جلو می‌ریم به (3,2) = W → هنوز داریم حریف می‌بینیم

جلو می‌ریم به (3,3) = W → باز هم حریف

جلو می‌ریم به (3,4) = B → دیدیم مهرهٔ خودی و قبلاً حداقل یک مهرهٔ حریف دیده بودیم → شرط (a) برقرار شد → جهت معتبر است.

چون حداقل یک جهت معتبر داشتیم، حرکت (3,0) معتبر است و اضافه می‌شود.

In [5]:
def valid_moves(board, player):
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]
    moves = []
    for x in range(8):
        for y in range(8):
            if board[x][y] != EMPTY:
                continue
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                found_opponent = False
                while 0 <= nx < 8 and 0 <= ny < 8:
                    if board[nx][ny] == -player:
                        found_opponent = True
                    elif board[nx][ny] == player and found_opponent:
                        moves.append((x, y))
                        break
                    else:
                        break
                    nx += dx
                    ny += dy
    return sorted(set(moves))


<div dir=rtl>
تابع زیر قرار است وضعیت صفحه را بعد از قرار دادن یک مهره بدست بیاورد.

ایکس و ایگرگ یکی از خانه هایی است که ما مثلا قبلا بررسی کردیم که میتوان مهره را در آن قرار داد. یه برد جدید میسازیم و در آن خانه ایکس و ایگرگ را برابر مقدار بازیکن قرار میدهیم.(به عبارت دیگر مهره بازیکن را آنجا قرار میدهیم.)

دوباره مانند تابع قبلی همسایه ها در هر جهت را به ان ایکس و ان ایگرگ میسازیم و بررسی میکنیم که در برد هستند یا نه.
حال یک لیست توفلیپ خالی میسازیم که قرار است به نمایش دهد  مهره های چه خانه هایی باید برگردند. 
سپس در حلقه وایل بررسی میشود اگر مهره همسایه متعلق به مخالف بود مهره به لیست اضافه شود و سراغ مهره همسایه بعدی میرویم تا جایی که  به مهره خودمان برسیم. 
در این حالت به ازای هر عضو لیست تو فلیپ خانه ها را متعلق به بازیکن میکنیم. 




In [6]:
def apply_move(board, move, player):
    x, y = move
    new_board = [row[:] for row in board] 
    new_board[x][y] = player

    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]

    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        to_flip = []
        while 0 <= nx < 8 and 0 <= ny < 8:
            if new_board[nx][ny] == -player:
                to_flip.append((nx, ny))
            elif new_board[nx][ny] == player:
                for fx, fy in to_flip:
                    new_board[fx][fy] = player
                break
            else:
                break
            nx += dx
            ny += dy
    return new_board


<div dir=rtl>
خب قبل از اینکه سراغ الگوریتم  مینیمکس برای طراحی قسمت هوش مصنوعی برویم نیاز داریم که امتیازات رو هم محاسبه کنیم. 


ینجا فرض شده BLACK = 1، WHITE = -1، و EMPTY = 0.

پس اگر یک خانه سیاه باشه → +1 به امتیاز کلی اضافه میشه.
اگر سفید باشه → -1 کم میشه.
اگر خالی باشه → 0 تغییر نمیده.

مراحل اجرا
score = 0 → مقدار اولیه.

دو حلقهٔ تو در تو:
در هر ردیف از صفحه و در هر خانه از صفحه 

score += cell : اضافه کردن مقدار (1 یا -1 یا 0) به جمع.

در آخر: player * score

این ضرب باعث میشه نتیجه نسبت به بازیکن جاری سنجیده بشه.

اگر player = BLACK (1)، نتیجه همون score میشه (مثبت یعنی برتری BLACK).

اگر player = WHITE (-1)، نتیجه معکوس میشه (مثبت یعنی برتری WHITE).

مثال
فرض کن score = 5 یعنی ۵ مهره سیاه بیشتر از سفید هست.

اگر player=BLACK : خروجی = 5 (خوب برای سیاه).

اگر player=WHITE : خروجی = -5 (بد برای سفید).

 و پایان بازی هم زمانی اتفاق می افتاد که هیچ حرکت معتبری برای بازیکن ها یافت نشود.

In [8]:
def evaluate(board, player):
    score = 0
    for row in board:
        for cell in row:
            score += cell
    return player * score


def game_over(board):
    return not valid_moves(board, BLACK) and not valid_moves(board, WHITE)



<div dir=rtl>
حال سراغ اصل الگوریتم میرویم. پیاده‌سازی الگوریتم مینیمکس با هرس آلفا-بتا برای انتخاب بهترین حرکت ممکن.

پارامتر های ما صفحه بازی هستند. عمق جستجو، آلف و بتا مقادیر محدود کننده برای هرس درخت، مقدار بولینی که نشان میدهد نوبت انسان است یا کامپیوتر.

متغیر ولید حرکت های معتبر را بسته به بازیکن در خودش ذخیره میکند و اگر چیزی یافت نشد  جست و جو رو متوقف میکنه و نتیجه بازی رو برمیگردونه.
در لایهٔ آخر (عمق صفر)، امتیاز هر حالت رو با evaluate داریم.
حال اگر در بازی نوبت کامپیوتر بود به ازای هر حرکت معتبر  موو را اعمال میکنیم. سپس تابع را برای  بازیکن انسان صدا بزن و نتیجه را در ایول ذخیره کن اگر نتیجه بهتر شد مقدار مکس ایول و بست مو را جا به جا کن و مقدار آلفا را برابر مکسیمم آلفا و ایول قرار بده. و اگر آلفا بزرگتر مساوی بتا شد حلقه را بشکن.

شاخه بعدی مربوط به بازیکن  انسان است و تفاوتش در مینیمایز کردن ایول است .

In [9]:

def minimax_ab(board, depth, alpha, beta, maximizing_player, player):
    if depth == 0 or game_over(board):
        return evaluate(board, player), None

    valid = valid_moves(board, player if maximizing_player else -player)

    if not valid:
        return evaluate(board, player), None

    if maximizing_player:
        max_eval = float('-inf')
        best_move = None
        for move in valid:
            new_board = apply_move(board, move, player)
            eval, _ = minimax_ab(new_board, depth-1, alpha, beta, False, player)
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        best_move = None
        for move in valid:
            new_board = apply_move(board, move, -player)
            eval, _ = minimax_ab(new_board, depth-1, alpha, beta, True, player)
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move



<div dir=rtl>
حال تابع زیر قرار است بازی را مدیریت کند. به این صورت که ابتدا برد اولیه را میسازد و بازیکن شروع کننده را سیاه قرار میدهد. سیاه همان انسان است.
ابتدا بررسی میشود که بازیکن ها حرکت دارند یا نه اگر هر کدام حرکت نداشتند نوبتشان به بازیکن دیگر داده میشود.

اگر نوبت بازیکن انسان بود یه ورودی  بین صفر تا هفت میگیرد برای سطر و ستون میگیرد و سپس  مهره را در این نقطه قرار میدهد. اگر امکان قرار گیری مهره در آن مکان نبود یا  ورودی نا معتبر بود اکسپشن به ما میدهد. 

اگر نوبت کامپیوتر بود تابع مینیمکس_اب صدا زده میشود و کامپیوتر بهترین حرکت را انتخاب  کرده و بازی میکند سپس نوبت بازیکن جا به جا میشود تا جایی که بازی تمام شود.

سپس تعداد مهره های هر بازیکن در صفحه شمرده میشود و بر اساس آن  اعلام میشود کامپیوتر برنده شده است یا انسان.


In [ ]:
def play_game():
    board = initial_board()
    current_player = BLACK 

    while not game_over(board):
        print_board(board)
        print()

        moves = valid_moves(board, current_player)
        if not moves:
            print("بازیکن", "○" if current_player == BLACK else "●", "حرکتی ندارد.")
            current_player *= -1
            continue

        if current_player == BLACK:
            print("نوبت شما (○). حرکت‌های مجاز:", moves)
            while True:
                try:
                    x = int(input("ردیف (0-7): "))
                    y = int(input("ستون (0-7): "))
                    if (x, y) in moves:
                        board = apply_move(board, (x, y), current_player)
                        break
                    else:
                        print("❌ حرکت نامعتبر. دوباره تلاش کن.")
                except:
                    print("❌ ورودی نامعتبر.")
        else:
            print("🤖 نوبت کامپیوتر (●)...")
            _, move = minimax_ab(board, 4, float('-inf'), float('inf'), True, WHITE)
            if move:
                print("کامپیوتر بازی کرد:", move)
                board = apply_move(board, move, WHITE)

        current_player *= -1
    
    print_board(board)
    black_score = sum(row.count(BLACK) for row in board)
    white_score = sum(row.count(WHITE) for row in board)
    print(f"\n🎯 نتیجه: ●={black_score}, ○={white_score}")
    if black_score > white_score:
        print("🏆 شما برنده شدید!")
    elif white_score > black_score:
        print("🤖 کامپیوتر برنده شد.")
    else:
        print("⚖️ بازی مساوی شد.")

play_game()



  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . ○ ● . . . 
4 . . . ● ○ . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 

نوبت شما (○). حرکت‌های مجاز: [(2, 3), (3, 2), (4, 5), (5, 4)]


<div dir=rtl>
حال بازی ما به صورت کنسولی کاملا آماده اجرا میباشد. برای اجرای کامل فایل eothello.py همراه با رابط گرافیکی پیاده سازی میکند.

در پیاده سازی رابط گرافیکی از کتاب خانه tkinter استفاده شده است
ابتدا صفحه کلی بازی طراحی میشود که یک ای آی دپس تعریف شده که با بالاتر بردن مقدار آن هوش مصنوعی عمیق تر درخت را میسازید و در نتیجه زمان پاسخگویی بیشتر و بازی سخت تر میشود.
بازیکن سفید کامپیوتر و سیاه انسان است. دکمه های بازی جدید و رد کردن (درصورتی که بازیکن انسان بازی نداشته باشد با کلیک روی دکمه تابع human pass اجرا میشود که نوبت را به هوش مصنوعی میدهد.) نیز تعریف شده اند.

تابع draw_disc بر اساس اینکه در خانه مهره سیاه یا سفید قرار دارد دایره میسازد. تابع draw_board برد را میسازد و به ازای حرکات ممکنی در تابع ولید موو محاسبه کردیم داخل آن خانه ها نقطه زرد رنگ قرار میدهد تا بازیکن راحت تر انتخاب کند. در این تابع با صدا زدن تابع draw_disc موقعیت مهره های قرار داده شده را در صفحه لود میکنیم.

تابع on_click وظیفه اش واکنش به کلیک بازیکن است. اگر نوبت کامپیوتر باشد هیچ کاری انجام نمیدهد. با کمک event مختصات را میگیرد و اگر در صفحه بازی بود به آی و جی تبدیل میکند. سپس بررسی میکند آیا قرار دادن مهره در این مکان مجاز هست یا خیر. اگر مجاز بود نوبت را عوض میکند سپس صفحه را دوباره رسم میکند.

تابع maybe_ai_move بررسی میکند که میتواند نوبت کامپیوتر باشد یا نه. اگر کامپیوتر در حال محاسبه بود کاری انجام نمیدهد در غیر این صورت اگر نوبت بازیکن سفید بود و هیچ حرکتی برای کامپیوتر وجود نداشت بازی را به انسان پاس میدهد(مشابه تابع human_pass)

تابع update_status نیز متنی که زیر صفحه از وضعیت بازی به بازیکن نشان میدهد را با تغیر وضعیت آپدیت میکند.


تابع run_ai_move نیز وظیفه اش انجام بازی از طرف کامپیوتر است. تابع minimax_ab را صدا میزند و پس از بدست آوردن مقدارش حرکت را انجام میدهد و  صفحه بازی را دوباره رسم میکند. اگر هم بازی تمام شده باشد نتیجه را نشان میدهد.

تابع show_result نیز تعداد مهره های هر کس داخل صفحه را میشمارد و در نهایت برنده را مشخص میکند و با باز کردن یک مسیج باکس نتیجه را نمایش میدهد.


